# 第4回講義 宿題

## 課題
今Lessonで学んだことを元に, MNISTのファッション版 (Fashion MNIST, クラス数10) を多層パーセプトロン (MLP) によって分類してみましょう.

Fashion MNISTの詳細については以下のリンクを参考にしてください.

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値
Accuracy: 85%

### ルール
- **下のセルで指定されている`x_train、y_train`以外の学習データは使わないでください。**
- **MLPのアルゴリズム部分の実装はnumpyのみで行ってください** (sklearnやtensorflowなどは使用しないでください)。
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません。

### 提出方法
- 2つのファイルを提出していただきます。
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し、**Homeworkタブから`chap04`を選択して**提出してください。
    2. それに対応するpythonのコードを`submission_code.py`として保存し、**Homeworkタブから`chap04 (code)`を選択して**提出してください。
      - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください（writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません。そのため、実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください）。
      
- なお、採点は1で行い、2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）。コードの内容を変更した場合は、**1と2の両方を提出し直してください**。

### 評価方法
- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します.
- 毎日夜24時にテストデータの一部に対する精度でLeader Boardを更新します.
- 締切日の夜24時にテストデータ全体に対する精度でLeader Boardを更新します. これを最終的な評価とします.

### データの読み込み
- この部分は修正しないでください。

In [1]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('../data/x_train.npy')
    y_train = np.load('../data/y_train.npy')
    
    # テストデータ
    x_test = np.load('../data/x_test.npy')

    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

### 多層パーセプトロンの実装

In [14]:
%%writefile submission_code.py

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

x_train, y_train, x_test = load_fashionmnist()

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=1000)

def np_log(x):
    return np.log(np.clip(x, 1e-10, x))

def relu(x):
    return np.maximum(x, 0)

def deriv_relu(x):
    return (x > 0).astype(x.dtype)

def softmax(x):
    x -= x.max(axis=1, keepdims=True)
    x_exp = np.exp(x)
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)

def deriv_softmax(x):
    return softmax(x) * (1 - softmax(x))


class Dense:
    
    def __init__(self, in_dim, out_dim, function, deriv_function):
        self.W = np.random.uniform(low=-0.08, high=0.08,size=(in_dim, out_dim)).astype('float64')
        self.b = np.zeros(out_dim).astype('float64')
        self.function = function
        self.deriv_function = deriv_function

        self.x = None
        self.u = None

        self.dW = None
        self.db = None

        self.params_idxs = np.cumsum([self.W.size, self.b.size])

    def __call__(self, x):
        self.x = x
        self.u = np.matmul(self.x, self.W) + self.b
        return self.function(self.u)

    def b_prop(self, delta, W):
        self.delta = self.deriv_function(self.u) * np.matmul(delta, W.T)
        return self.delta
    
    def compute_grad(self):
        batch_size = self.delta.shape[0]
        self.dW = np.matmul(self.x.T, self.delta) / batch_size
        self.db = np.matmul(np.ones(batch_size), self.delta) / batch_size

    def get_params(self):
        return np.concatenate([self.W.ravel(), self.b], axis=0)
    
    def set_params(self, params):
        _W, _b = np.split(params, self.params_idxs)[:-1]
        self.W = _W.reshape(self.W.shape)
        self.b = _b
    
    def get_grads(self):
        return np.concatenate([self.dW.ravel(), self.db], axis=0)

    
def f_props(layers, x):
    for layer in layers:
        x = layer(x)
    return x

def b_props(layers, delta):
    batch_size = delta.shape[0]
    
    for i, layer in enumerate(layers[::-1]):
        if i == 0: # 出力層の場合
            layer.delta = delta # y - t
            layer.compute_grad() # 勾配の計算
        else: # 出力層以外の場合
            delta = layer.b_prop(delta, W) # 逆伝播
            layer.compute_grad() # 勾配の計算

        W = layer.W

def update_parameters(layers, eps):
    for layer in layers:
        layer.W -= eps * layer.dW
        layer.b -= eps * layer.db

layers = [
    Dense(784, 100, relu, deriv_relu),
    Dense(100, 100, relu, deriv_relu),
    Dense(100, 10, softmax, deriv_softmax)
]

def train_mnist(x, t, eps):
    # 順伝播
    y = f_props(layers, x)
    
    # 誤差の計算
    cost = (- t * np_log(y)).sum(axis=1).mean()
    
    # 逆伝播
    delta = y - t
    b_props(layers, delta)

    # パラメータの更新
    update_parameters(layers, eps)

    return cost

def valid_mnist(x, t):
    # 順伝播
    y = f_props(layers, x)
    
    # 誤差の計算
    cost = (- t * np_log(y)).sum(axis=1).mean()
    
    return cost, y

#学習
batch_size = 10
for epoch in range(10):
    x_train, y_train = shuffle(x_train, y_train)
    for i in range(int(len(x_train) / batch_size)):
        x = x_train[i * batch_size : (i+1) * batch_size]
        y = y_train[i * batch_size : (i+1) * batch_size]
        cost = train_mnist(x, y, eps = 0.01)
    
    cost, y_pred = valid_mnist(x_valid, y_valid)
    accuracy = accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(epoch + 1, cost, accuracy))

#テストデータの分類
y_pred = f_props(layers, x_test)
y_pred = np.argmax(y_pred, axis=1)

submission = pd.Series(y_pred, name='label')
submission.to_csv('submission_pred.csv', header=True, index_label='id')

Writing submission_code.py
